<a href="https://colab.research.google.com/github/smit-r/Shape-Generation-using-Spatially-Partitioned-Point-Clouds/blob/main/KDTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install open3d

In [ ]:
import numpy as np
import scipy as scy
import os
import open3d
import numpy as np
import sklearn.decomposition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/shapenet_chair_data/shapenet-chairs-pcd

/content/drive/My Drive/Colab Notebooks/shapenet_chair_data/shapenet-chairs-pcd


#### KDTree

In [ ]:
## performs kdtree sorting on pointcloud
def kdtree(pointCloud, level):
  if (len(pointCloud)<=1):
    return pointCloud
  pointlist = np.array(pointCloud)
  plane = getPlane(level)
  median = getkthPosition(pointlist, plane, 0, len(pointlist)-1, int(len(pointlist)/2))
  leftArray = median[:int(len(median)/2)]
  rightArray = median[int((len(median)/2))+1:]
  leftSortedArray = kdtree(leftArray, level+1)
  rightSortedArray = kdtree(rightArray, level+1)
  return np.concatenate((leftSortedArray, [median[int(len(median)/2)]], rightSortedArray))

In [ ]:
def getPlane(level):
  plane = level%3
  return plane

In [ ]:
def getPartition(pointlist, axis, l, r):
  i = l
  x = pointlist[r, axis]
  for j in range(l, r):
    if pointlist[j,axis]<x:
      pointlist[[i,j]] = pointlist[[j,i]]
      i += 1
  pointlist[[i,r]] = pointlist[[r,i]]
  return i

In [ ]:
def getkthPosition(pointlist, axis, l, r, k):
  i = getPartition(pointlist, axis, l, r)
  if (l>r):
    return np.array([])
  if (i == k):
    return pointlist
  elif (i == r):
    return getkthPosition(pointlist, axis, l, r-1, k)
  elif (i < k):
    return getkthPosition(pointlist, axis, i+1, r, k)
  else:
    return getkthPosition(pointlist, axis, l, i-1, k)

#### PCA Analysis

In [ ]:
## load the dataset
dataset = []
for f in os.listdir():
  if f[-4:]==".pcd":
    pcd = open3d.io.read_point_cloud(f)
    pointcloud = np.asarray(pcd.points)
    dataset.append(pointcloud)
  print(f)

In [ ]:
# sort using KDTree
data = np.array(dataset)
for i in range(len(dataset)):
  shape = data[i]
  shape = kdtree(shape, 0)
  data[i] = shape
  print(i)

In [ ]:
# construct matrix P
P = np.reshape(data, [5000,-1]) #(5000, 3000)
mean = np.mean(P, axis=0) #(5000, )
Pu = P - mean #P[i]-mean
Pu.shape

(5000, 3000)

In [ ]:
# PCA
def pcaAnalysis(Pu):
  pca = sklearn.decomposition.PCA(n_components=100)
  pca.fit(Pu)
  comp = pca.components_
  return comp

comp = pcaAnalysis(Pu)

In [ ]:
# transform and reconstruct
Pcomp = np.dot(Pu, comp.T)
print(Pcomp.shape)
Precon = np.dot(Pcomp, comp)
print(Precon.shape)
errors = Pu - Precon
errors = errors*errors #calculate errors for each shape
print(errors.shape)
errors = np.sum(errors, axis=1) #(5000, )
print(errors.shape)
errlist = [errors]

(5000, 100)
(5000, 3000)
(5000, 3000)
(5000,)


In [ ]:
# sample two points randomly
def samplePoints():
  points = np.random.randint(0,1000, 2)
  return points

In [ ]:
# swap points
def swap(shape, points):
  shape[3*points[0]], shape[3*points[1]] = shape[3*points[1]], shape[3*points[0]] # swap x
  shape[3*points[0]+1], shape[3*points[1]+1] = shape[3*points[1]+1], shape[3*points[0]+1] #swap y
  shape[3*points[0]+2], shape[3*points[1]+2] = shape[3*points[1]+2], shape[3*points[0]+2] #swap z
  return shape

In [ ]:
# calculate error
def error(shape, shape_):
  err = shape - shape_
  err = np.dot(err.T, err)
  return err

In [ ]:
# ordering using PCA Reconstruction Error
# as given in paper
save_path = '/content/drive/My Drive/'
k = 10**4
count = 0
errs = errlist[0][0]
for s in range(len(Pu)):
  for i in range(k):
    shape = np.array(Pu[s])
    points = samplePoints() # sample two points
    shape = swap(shape, points)
    shapecomp = np.dot(shape.T, comp.T) # transform
    shaperecon = np.dot(shapecomp.T, comp) # inverse_transform
    err = error(shape, shaperecon)
    if err<errs:
      Pu[s] = shape
      count += 1
    else:
      continue
  print(count, s)
  comp = pcaAnalysis(Pu) # calculate new components
  #calculate error for next shape
  if s<len(Pu-1):
    shape = Pu[s+1]
    shapecomp = np.dot(shape.T, comp.T)
    shaperecon = np.dot(shapecomp.T, comp)
    errs = error(shape, shaperecon)
  if s%50==0 or s==len(Pu)-1: #calculate errors for each shape and store in errlist
    Pcomp = np.dot(Pu, comp.T)
    Precon = np.dot(Pcomp, comp)
    errors = Pu - Precon
    errors = errors*errors
    errors = np.sum(errors, axis=1)
    errlist.append(errors)
    np.save(save_path+'errlist', np.array(errlist))
    np.save(save_path+'Pu', Pu)

## results after performing swaping for 1000 shapes

[Reconstruction Error](https://drive.google.com/file/d/1MGAZNpFGKdh0b-mEuOIDiWvut3WGs_vA/view?usp=sharing)

In [ ]:
def swapAll(Pu, Points):
  for i in range(len(points)):
    Pu[i] = swap(Pu[i], points[i])
  return Pu

In [ ]:
def sampleAll():
  points = np.random.randint(0,1000,(5000,2))
  return points

In [ ]:
def errAll(Pu_, comp):
  Pcomp = np.dot(Pu_, comp.T)
  Precon = np.dot(Pcomp, comp)
  errors = Pu_ - Precon
  errors = errors*errors
  errors = np.sum(errors, axis=1)
  return errors

In [ ]:
#### optimized trainig loop
save_path = '/content/drive/My Drive/'
for iter in range(1000):
  for k in range(10*4):
    Pu_ = np.array(Pu)
    points = sampleAll()
    Pu_ = swapAll(Pu_, points)
    # calculate error
    err = errAll(Pu_, comp)
    # compare
    bin = (err<errors).astype(int)
    binPu = np.array([bin]*3000).T
    Pu = ((1-binPu)*Pu) + (binPu*Pu_) # select shapes to be swaped and swap them
  comp = pcaAnalysis(Pu)
  errors = errAll(Pu, comp)
  errlist.append(errors)
  np.save(save_path+'vec_errlist', np.array(errlist))
  np.save(save_path+'vec_Pu', Pu)
  print(iter, np.sum(errors))

0 19569.186676428057
1 19551.682837160115
2 19534.242262590767
3 19517.54166685394
4 19499.15250251531
5 19481.24865742768
6 19461.755500221647
7 19443.155743331066
8 19426.312453648843
9 19408.023807837657
10 19391.9302040219
11 19374.626828593668
12 19359.42740421809
13 19340.48563459366
14 19323.222387808884
15 19307.179904682227
16 19289.406884801683
17 19274.298867871476
18 19256.775584632076
19 19241.893916388755
20 19224.51585749694
21 19208.877942698113
22 19192.2439250288
23 19175.792603144146
24 19159.745262332555
25 19144.023659963445
26 19128.219691463135
27 19112.40878709018
28 19095.92056188584
29 19081.42357771124
30 19064.46574667252
31 19049.030930089324
32 19033.560697122375
33 19018.273393434436
34 19003.419103827797
35 18988.29438052002
36 18972.776709414433
37 18957.19772639135
38 18941.916563718325
39 18926.90538135384
40 18912.358405146922
41 18896.91853771838
42 18881.97412656741
43 18866.7176170365
44 18851.540123979612
45 18836.89720453243
46 18822.94732695696

## results after 400 iterations

[Reconstruction Error](https://drive.google.com/file/d/1-46E_roxPaFlTNoULcBjBYmD3uCfdoRH/view?usp=sharing)

#### GAN 

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
class Generator(layers.Layer):

    def __init__(self, dim=100, name="Generator", **kwargs):
        super(Generator, self).__init__(name=name, **kwargs)
        self.layer1 = layers.Dense(dim, activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.layer2 = layers.Dense(dim, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.layer3 = layers.Dense(dim, activation='relu')
        self.bn3 = layers.BatchNormalization()
        self.layer4 = layers.Dense(dim, activation='relu')
        self.bn4 = layers.BatchNormalization()

    def call(self, inputs):
        x = self.layer1(inputs)
        x = self.bn1(x)
        x2 = self.layer2(x)
        x2 = self.bn2(x2)
        x3 = self.layer3(x2)
        x3 = self.bn3(x3)
        x4 = self.layer4(x3)
        x4 = self.bn4(x4)
        return x4

In [ ]:
class Discriminator(layers.Layer):

    def __init__(self, dim=100, name="Discriminator", **kwargs):
        super(Discriminator, self).__init__(name=name, **kwargs)
        self.layer1 = layers.Dense(dim, activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.layer2 = layers.Dense(dim, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.layer3 = layers.Dense(dim, activation='relu')
        self.bn3 = layers.BatchNormalization()
        self.layer4 = layers.Dense(dim, activation='relu')
        self.bn4 = layers.BatchNormalization()
        self.outlayer = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x1 = self.layer1(inputs)
        x1 = self.bn1(x1)
        x2 = self.layer2(x1)
        x2 = self.bn2(x2)
        x3 = self.layer3(x2)
        x3 = self.bn3(x3)
        x4 = self.layer4(x3)
        x4 = self.bn4(x4)
        pred = self.outlayer(x4)
        return x1,x2,x3,x4,pred

In [ ]:
def gen_loss_fn(fake, real):
  # expectation term
  lossE = 0
  lossCov = 0
  for i in range(len(real)):
    lossE += tf.math.reduce_sum(tf.math.square(tf.math.reduce_mean(real[i], axis=0) - tf.math.reduce_mean(fake[i], axis=0)))
    covfx = tf.linalg.matmul((real[i] - tf.math.reduce_mean(real[i], axis=0)), (real[i] - tf.math.reduce_mean(real[i], axis=0)), transpose_a=True, transpose_b=False)
    covfgx = tf.linalg.matmul((fake[i] - tf.math.reduce_mean(fake[i], axis=0)), (fake[i] - tf.math.reduce_mean(fake[i], axis=0)), transpose_a=True, transpose_b=False)
    lossCov += tf.math.square(tf.norm(covfx - covfgx))
  return lossE + lossCov

In [ ]:
## training loop
gen = Generator(dim=100)
dis = Discriminator(dim=100)

gen_opt = keras.optimizers.Adam()
dis_opt = keras.optimizers.Adam()

d_loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

@tf.function
def train_step(shape_coeff):
  ## sample noise vectors
  noise = tf.random.uniform(shape=shape_coeff.shape, minval=-1, maxval=1) ## (batch_size, 100)
  ## get generator output
  gen_shape_coeff = gen(noise) ## (batch_size, 100)
  comb_shape_coeff = tf.concat([gen_shape_coeff, shape_coeff], axis=0) ## (2*batch_size, 100)
  ## generate labels
  labels = tf.concat([tf.ones(gen_shape_coeff.shape[0],1), tf.zeros(shape_coeff.shape[0],1)], axis=0) ## (2*batch_size, 1)
  labels += 0.05*tf.random.uniform(labels.shape) 

  ## train the discriminator
  with tf.GradientTape() as tape:
    pred = dis(comb_shape_coeff)[-1] ## (2*batch_size, 1)
    d_loss = d_loss_fn(labels, pred)
  grads = tape.gradient(d_loss, dis.trainable_weights)
  dis_opt.apply_gradients(zip(grads, dis.trainable_weights))

  #sample noise for gen training
  noise_g = tf.random.uniform(shape=shape_coeff.shape, minval=-1, maxval=1) ## (batch_size, 100)
  #get discriminator output for real data
  dis_output = dis(shape_coeff) ## (x1,x2,x3,x4,pred)

  ## train the generator
  with tf.GradientTape() as tape:
    pred_g = dis(gen(noise_g)) ## (x1,x2,x3,x4,pred)
    g_loss = gen_loss_fn(dis_output, pred_g)
  grads = tape.gradient(g_loss, gen.trainable_weights)
  gen_opt.apply_gradients(zip(grads, gen.trainable_weights))
  return d_loss, g_loss, gen_shape_coeff

In [ ]:
Pcomp = np.dot(Pu, comp.T )
Pcomp = Pcomp.astype('float32')

In [ ]:
## set betch_Size and epochs here and run this cell for training

batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(Pcomp)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)


epochs = 1
save_dir = ''


for epoch in range(epochs):
  print("\nstart epoch", epoch)
  for step, batch in enumerate(dataset):
    # train step
    d_loss, g_loss, gen_shape_coeff = train_step(batch)

    #logging
    if step%50==0:
      print("discriminator loss at step %d: %.2f" % (step, d_loss))
      print("generator loss at step %d: %.2f" % (step, g_loss))


      # save weights and gen_outputs


start epoch 0
discriminator loss at step 0: 0.72
generator loss at step 0: 7713.96
discriminator loss at step 50: 0.49
generator loss at step 50: 297497.19
discriminator loss at step 100: 0.49
generator loss at step 100: 396538.44
discriminator loss at step 150: 0.49
generator loss at step 150: 450521.41
